# HW4 Runner

In [1]:
import papermill as pm
import scrapbook as sb
import pandas as pd
from datetime import datetime, timedelta
import os
import sqlite3


/Users/alexmccall/opt/anaconda3/envs/mfe-preprogram4/lib/python3.8/site-packages/papermill/iorw.py:50: FutureWarning: pyarrow.HadoopFileSystem is deprecated as of 2.0.0, please use pyarrow.fs.HadoopFileSystem instead.
  from pyarrow import HadoopFileSystem


get token list

In [2]:
connection_string = 'sqlite:///data/data.db'

In [3]:
# FullDB = pd.read_sql(f'''
# SELECT
#     *
# FROM ohlc
# ''',connection_string)
# # group by token
# FullDB

In [4]:
tokenData = pd.read_sql(f'''
SELECT
    token
FROM ohlc
group by token
''',connection_string)
tokenData

,token
0,AAVE
1,ADA
2,ATOM
3,AVAX
4,BTC
5,COMP
6,CRV
7,DOT
8,ETH
9,SOL


In [5]:
tokenList = tokenData['token'].to_list()
tokenList

['AAVE',
 'ADA',
 'ATOM',
 'AVAX',
 'BTC',
 'COMP',
 'CRV',
 'DOT',
 'ETH',
 'SOL',
 'USDT']

papermill

get start date for last 2 complete weeks

In [6]:
def last_dow(d: datetime, dow: int) -> datetime:
    '''
    Gets the date of the last day of week, using datetime convention of Monday is 0 and Sunday is 6
    '''
    dow_d = d.weekday()
    days_delta = (7 - dow) + dow_d if dow_d < dow else dow_d - dow
    return d - timedelta(days=days_delta)

In [7]:
ndays = 14
startDate = last_dow(datetime(2021, 12, 15), 4) - timedelta(days=ndays)
startDate

datetime.datetime(2021, 11, 26, 0, 0)

set parameters

In [8]:
run_dates = [(startDate + timedelta(days=ndays * x)) for x in range(1)]
tokens = tokenList
run_dates

[datetime.datetime(2021, 11, 26, 0, 0)]

create folder for output runs

In [9]:
base_output_folder = os.path.expanduser('outputs')

if not os.path.exists(base_output_folder):
        os.makedirs(base_output_folder)

In [10]:
for run_date in run_dates:
    for token in tokens:
        print(f'running for {token} for date {run_date} to {run_date + timedelta(days=14)}', end='\r')
        res = pm.execute_notebook(
            'hw_4_alex_mccall_template.ipynb',
            f'{base_output_folder}/market_analysis_{token}_{run_date}.ipynb',
            parameters = {
                'from_date': f'{run_date}',
                'to_date': f'{run_date + timedelta(days=ndays)}',
                'token': token,
                'connection_string': connection_string,
                # 'events_db': events_db
            }
        )

Executing:   0%|          | 0/16 [00:00<?, ?cell/s]

Executing:   0%|          | 0/16 [00:00<?, ?cell/s]

Executing:   0%|          | 0/16 [00:00<?, ?cell/s]

Executing:   0%|          | 0/16 [00:00<?, ?cell/s]

Executing:   0%|          | 0/16 [00:00<?, ?cell/s]

...

Executing:   0%|          | 0/16 [00:00<?, ?cell/s]

Executing:   0%|          | 0/16 [00:00<?, ?cell/s]

Executing:   0%|          | 0/16 [00:00<?, ?cell/s]

Executing:   0%|          | 0/16 [00:00<?, ?cell/s]

Executing:   0%|          | 0/16 [00:00<?, ?cell/s]

collect data from notebooks and analyze

In [11]:
nbs = sb.read_notebooks('outputs')

In [12]:
summary = pd.DataFrame([
    pd.Series({
        'token': scrap['token'].data,
        'from_date': scrap['from_date'].data,
        'to_date': scrap['to_date'].data,
        # 'avg_hrly_return': scrap['prices'].data.return.mean(),
        'avg_hrly_return': scrap['prices'].data['return'].mean(),
        'avg_hrly_volatility': scrap['prices'].data.volatility.mean(),
        'max_drawdown': scrap['prices'].data.maxdrawdown.iloc[-1],
        'range_lastclose-firstclose': scrap['prices'].data.close.iloc[-1] - scrap['prices'].data.close.iloc[0]
    }) 
    for book, scrap in nbs.notebook_scraps.items()
])
summary

,token,from_date,to_date,avg_hrly_return,avg_hrly_volatility,max_drawdown,range_lastclose-firstclose
0,AAVE,2021-11-26 00:00:00,2021-12-10 00:00:00,-1.220332e-03,0.214692,-0.365927,-83.7200
1,ADA,2021-11-26 00:00:00,2021-12-10 00:00:00,-8.135194e-04,0.215395,-0.270810,-0.3681
2,ATOM,2021-11-26 00:00:00,2021-12-10 00:00:00,-9.327062e-04,0.300376,-0.318250,-6.9340
3,AVAX,2021-11-26 00:00:00,2021-12-10 00:00:00,-1.117644e-03,0.306359,-0.385790,-32.7900
4,BTC,2021-11-26 00:00:00,2021-12-10 00:00:00,-6.503661e-04,0.152091,-0.210191,-11017.6200
5,COMP,2021-11-26 00:00:00,2021-12-10 00:00:00,-1.235754e-03,0.226777,-0.333968,-91.9800
6,CRV,2021-11-26 00:00:00,2021-12-10 00:00:00,-1.623725e-03,0.356971,-0.455827,-2.2364
7,DOT,2021-11-26 00:00:00,2021-12-10 00:00:00,-1.206551e-03,0.248847,-0.357445,-12.2390
8,ETH,2021-11-26 00:00:00,2021-12-10 00:00:00,-2.879175e-04,0.175245,-0.188101,-357.3800
9,SOL,2021-11-26 00:00:00,2021-12-10 00:00:00,-4.896815e-04,0.249007,-0.261363,-26.3790
